In [1]:
import numpy as np
import pandas as pd

In [2]:
ds = pd.read_csv('cleaned_data.csv')

In [3]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 724 entries, 0 to 723
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  724 non-null    int64 
 1   level_0     724 non-null    int64 
 2   index       724 non-null    int64 
 3   name        724 non-null    object
 4   company     724 non-null    object
 5   year        724 non-null    int64 
 6   Price       724 non-null    int64 
 7   kms_driven  724 non-null    int64 
 8   fuel_type   724 non-null    object
dtypes: int64(6), object(3)
memory usage: 51.0+ KB


In [4]:
columns = ["company", "name", "year", "kms_driven", "fuel_type"]
X = ds[columns]
y = ds[["Price"]]

In [12]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
ohe = OneHotEncoder()
ohe.fit(X[["company", "name", "fuel_type"]])

OneHotEncoder()

In [14]:
ct = make_column_transformer((OneHotEncoder(categories = ohe.categories_), 
                              ["company", "name", "fuel_type"]), 
                             remainder = 'passthrough')

In [16]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()

In [17]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(ct, reg)

In [18]:
pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
       'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Vol...
       'i10 Sportz', 'i10 Sportz 1.2', 'i20', 'i20 Active',
       'i20 Active 1.2 SX', 'i20 Active 1.4L SX O', 'i20 Asta',
       'i20 Asta 1.2', 'i20 Asta 1.4 CRDI 6 Speed', 'i20 Magna',
       'i20 Magna 1.2', 'i20 Magna O 1.2', 'i20 Select Variant',
       'i20 Sportz 1.2', 'i20 Sportz 1.4 CRDI'], dtype=object),
                                                                            array(['Diesel', 'LPG', 'Petrol'], dtype=object)]),
                                                  ['company', 'name',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

In [48]:
#Divide dataset into training and testing
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

scores = []

for i in range(0, 1000):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, random_state = i)
    pipe.fit(X_train, y_train)
    result = pipe.predict(X_test)
    score = r2_score(y_test, result)
    scores.append(score)

In [52]:
best = np.argmax(scores)

In [29]:
# linear_model = pipe.named_steps['linearregression']

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, random_state = best)
pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
       'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Vol...
       'i10 Sportz', 'i10 Sportz 1.2', 'i20', 'i20 Active',
       'i20 Active 1.2 SX', 'i20 Active 1.4L SX O', 'i20 Asta',
       'i20 Asta 1.2', 'i20 Asta 1.4 CRDI 6 Speed', 'i20 Magna',
       'i20 Magna 1.2', 'i20 Magna O 1.2', 'i20 Select Variant',
       'i20 Sportz 1.2', 'i20 Sportz 1.4 CRDI'], dtype=object),
                                                                            array(['Diesel', 'LPG', 'Petrol'], dtype=object)]),
                                                  ['company', 'name',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

In [35]:
pipe.predict()

0.715144499232695

In [60]:
myinput = pd.DataFrame(columns = columns, data = [["Tata", "Indigo LX TDI BS III", 2015, 100000, "Diesel"]])
result = round(pipe.predict(myinput)[0,0])
result

314386

In [61]:
import pickle as pkl
pkl.dump(pipe, open("CPP.pkl", "wb"))